## Agenda

This notebook is meant to prepare the data to be used for regression analysis.<br>

The following steps will be performed:<br>
    1. Pull Divvy trip data into one DataFrame
    2. Visualize patterns of trip frequency
    3. Set a 'time_bucket' variable
    4. Create a .csv of trip_data for future use
    5. Create a DataFrame shell for aggregating trip data and weather data
    6. Add in trip data
    6. Add in weather data
    7. Clean up and export to .csv

In [1]:
# Start out by importing necessary software packages.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
%matplotlib inline

### Pull Divvy trip data into one DataFrame

In [2]:
# Column names have varied slightly over time, so we need to set them ourselves for consistency.
columns = [
    'trip_id',
    'start_time',
    'end_time',
    'bikeid',
    'tripduration',
    'from_station_id',
    'from_station_name',
    'to_station_id',
    'to_station_name',
    'usertype',
    'gender',
    'birthyear'
]

# To ensure quality, we will specify data types.
# Note, the birthyear has to be a float (not an int) because it can contain null values. 
data_types = {
    'trip_id':np.int64,
    'start_time':object,
    'end_time':object,
    'bikeid':np.int64,
    'tripduration':np.int64,
    'from_station_id':np.int64,
    'from_station_name':object,
    'to_station_id':np.int64,
    'to_station_name':object,
    'usertype':object,
    'gender':object,
    'birthyear':np.float64
}

# The first data file, 'Divvy_Trips_2013.csv', is omitted from the file list, as it is used to initialize the DataFrame.
file_names = [
    'Divvy_Trips_2014_Q1Q2.csv',
    'Divvy_Trips_2014-Q3-07.csv',
    'Divvy_Trips_2014-Q3-0809.csv',
    'Divvy_Trips_2014-Q4.csv',
    'Divvy_Trips_2015-Q1.csv',
    'Divvy_Trips_2015-Q2.csv', 
    'Divvy_Trips_2015_07.csv',
    'Divvy_Trips_2015_08.csv',
    'Divvy_Trips_2015_09.csv',
    'Divvy_Trips_2015_Q4.csv',
    'Divvy_Trips_2016_Q1.csv',
    'Divvy_Trips_2016_04.csv',
    'Divvy_Trips_2016_05.csv',
    'Divvy_Trips_2016_06.csv',
    'Divvy_Trips_2016_Q3.csv',
    'Divvy_Trips_2016_Q4.csv',   
    'Divvy_Trips_2017_Q1.csv',
    'Divvy_Trips_2017_Q2.csv',
    'Divvy_Trips_2017_Q3.csv',
    'Divvy_Trips_2017_Q4.csv'
]

# We initialize the DataFrame by loading in the first datafile.
# Then we remove any duplicates.
# Finally, we convert the start_time and end_time columns into datetimes
raw_trip_data = pd.read_csv('./Raw_Data/Divvy/Unzipped/Trips/Divvy_Trips_2013.csv',
                            header=0, names=columns, index_col='trip_id', dtype=data_types,
                            parse_dates=['start_time','end_time'], infer_datetime_format=True)
raw_trip_data = raw_trip_data.drop_duplicates(keep='first')

# We iterate through the file names and append to the raw data file, again removing duplicates and converting to datetime.
for file_name in file_names:
    file_path = './Raw_Data/Divvy/Unzipped/Trips/'+str(file_name)
    new_data = pd.read_csv(file_path, header=0, names=columns, index_col='trip_id', dtype=data_types,
                          parse_dates=['start_time','end_time'], infer_datetime_format=True)
    new_data = new_data.drop_duplicates(keep='first')
    raw_trip_data = raw_trip_data.append(new_data, verify_integrity=True)

raw_trip_data = raw_trip_data.drop_duplicates(keep='first')

# We drop unnecessary columns to create the initial 'trip_data' DataFrame.
unnecessary_columns = ['bikeid','tripduration','from_station_name','to_station_name','usertype','gender','birthyear']
trip_data = raw_trip_data.drop(unnecessary_columns, axis=1)

C:\Users\jiconway\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# We add separate columns in 'trip_data' for each unit of time that we will need.

trip_data.loc[:,'start_date'] = trip_data.loc[:,'start_time'].dt.date
trip_data.loc[:,'start_weekday'] = trip_data.loc[:,'start_time'].dt.weekday
trip_data.loc[:,'start_hour'] = trip_data.loc[:,'start_time'].dt.hour

trip_data.loc[:,'end_date'] = trip_data.loc[:,'end_time'].dt.date
trip_data.loc[:,'end_weekday'] = trip_data.loc[:,'end_time'].dt.weekday
trip_data.loc[:,'end_hour'] = trip_data.loc[:,'end_time'].dt.hour

### Visualize patterns of trip frequency

In [4]:
# Next, we get the count of trips by day and hour (based on start_time).
# These results were then visually analyzed in Excel.
pd.crosstab(trip_data.loc[:,'start_hour'], trip_data.loc[:,'start_weekday'])

start_weekday,0,1,2,3,4,5,6
start_hour,,,,,,,
0,11594,8975,9553,10616,12795,22397,27008
1,6096,4592,4786,5647,7181,16325,17777
2,3475,2420,2450,2860,4064,10147,12030
3,2222,1557,1759,1816,2372,5230,6879
4,3210,3407,3174,3174,3490,3109,4308
5,16081,18972,18347,17476,16138,5064,4771
6,60490,69503,68590,63285,58666,11618,9871
7,137481,154847,150734,140523,134921,27492,21355
8,176038,194270,188089,181249,176755,53627,42142


In [5]:
# We repeat the process using end_time to confirm that the patterns hold, which they do.
pd.crosstab(trip_data.loc[:,'end_hour'], trip_data.loc[:,'end_weekday'])

end_weekday,0,1,2,3,4,5,6
end_hour,,,,,,,
0,13905,10641,11539,12344,14663,24601,30385
1,7114,5359,5508,6465,8104,17201,19968
2,4410,3030,3045,3558,4938,12083,13466
3,2581,1788,1936,1992,2586,6103,8261
4,2999,2823,2735,2759,3220,3659,4807
5,13155,15621,14953,14521,13377,4273,4460
6,50779,58326,57543,52956,49377,9812,8288
7,119109,135284,132777,123268,117299,22585,16974
8,178608,199030,192349,184211,178787,46640,35359


### Set time buckets

Based on the results of the visual analysis, the following time buckets were determined:<br>
    0. Weekday = 0-4, Hour = 0-5 (1.52% of trips)
    1. Weekday = 0-4, Hour = 6-9 (17.26% of trips)
    2. Weekday = 0-4, Hour = 10-15 (19.85% of trips)
    3. Weekday = 0-4, Hour = 16-19 (26.49% of trips)
    4. Weekday = 0-4, Hour = 20-23 (6.84% of trips)
    5. Weekday = 5-6, Hour = 0-9 (3.35% of trips)
    6. Weekday = 5-6, Hour = 10-18 (20.62% of trips)
    7. Weekday = 5-6, Hour = 19-23 (4.08% of trips)

Notes:<br>
Percentages based on start_time (similar results for end_time).<br>
See supporting workfile 'Trips by Weekday and Hour.xlsx' for visualization.

In [6]:
# We define a function to set the time bucket for the start_time, then use .apply() to add it to the data.
def set_start_time_bucket(row):
    if row.loc['start_weekday'] <= 4:
        if row.loc['start_hour'] < 6:
            return 0
        elif row.loc['start_hour'] < 10:
            return 1
        elif row.loc['start_hour'] < 16:
            return 2
        elif row.loc['start_hour'] < 20:
            return 3
        else:
            return 4
    else:
        if row.loc['start_hour'] < 10:
            return 5
        elif row.loc['start_hour'] < 19:
            return 6
        else:
            return 7

trip_data.loc[:,'start_time_bucket'] = trip_data.apply(set_start_time_bucket, axis=1)

# We do the same process for end_time.
def set_end_time_bucket(row):
    if row.loc['end_weekday'] <= 4:
        if row.loc['end_hour'] < 6:
            return 0
        elif row.loc['end_hour'] < 10:
            return 1
        elif row.loc['end_hour'] < 16:
            return 2
        elif row.loc['end_hour'] < 20:
            return 3
        else:
            return 4
    else:
        if row.loc['end_hour'] < 10:
            return 5
        elif row.loc['end_hour'] < 19:
            return 6
        else:
            return 7

trip_data.loc[:,'end_time_bucket'] = trip_data.apply(set_end_time_bucket, axis=1)

### Create a csv of trip_data

In [7]:
# We create a text 'tag' that aggregates station ID, date, and time bucket.
trip_data.loc[:,'start_tag'] = 'ID' + trip_data.loc[:,'from_station_id'].astype(str) + 'Date' + trip_data.loc[:,'start_date'].astype(str) + 'Time' + trip_data.loc[:,'start_time_bucket'].astype(str)
trip_data.loc[:,'end_tag'] = 'ID' + trip_data.loc[:,'to_station_id'].astype(str) + 'Date' + trip_data.loc[:,'end_date'].astype(str) + 'Time' + trip_data.loc[:,'end_time_bucket'].astype(str)

In [9]:
# We are going to create a csv of the trip_data so that we can avoid running the cells above again if the kernel restarts.
# We need the data types to turn the csv back into a DataFrame.
trip_data.dtypes

start_time           datetime64[ns]
end_time             datetime64[ns]
from_station_id               int64
to_station_id                 int64
start_date                   object
start_weekday                 int64
start_hour                    int64
end_date                     object
end_weekday                   int64
end_hour                      int64
start_time_bucket             int64
end_time_bucket               int64
start_tag                    object
end_tag                      object
dtype: object

In [10]:
# We create the csv of trip_data to call back later.
trip_data.to_csv('./Supporting_Workfiles/trip_data.csv')

In [2]:
# We call back the trip_data csv.
# If the kernel restarts, RUN THIS CELL FIRST, DISREGARD CELLS ABOVE (except for the initial cell to import packages).
columns = [
    'trip_id',
    'start_time',
    'end_time',
    'from_station_id',
    'to_station_id',
    'start_date',
    'start_weekday',
    'start_hour',
    'end_date',
    'end_weekday',
    'end_hour',
    'start_time_bucket',
    'end_time_bucket',
    'start_tag',
    'end_tag'
]

data_types = {
    'trip_id':np.int64,
    'start_time':object,
    'end_time':object,
    'from_station_id':np.int64,
    'to_station_id':np.int64,
    'start_date':object,
    'start_weekday':np.int64,
    'start_hour':np.int64,
    'end_date':object,
    'end_weekday':np.int64,
    'end_hour':np.int64,
    'start_time_bucket':np.int64,
    'end_time_bucket':np.int64,
    'start_tag':object,
    'end_tag':object
}

trip_data = pd.read_csv('./Supporting_Workfiles/trip_data.csv',
                            header=0, names=columns, index_col='trip_id', dtype=data_types,
                            parse_dates=['start_time','end_time'], infer_datetime_format=True)

C:\Users\jiconway\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Create a DataFrame shell for aggregating trip data and weather data

In [23]:
# We need some information about the stations to combine with data provided by Divvy in Excel.

unique_stations = pd.unique(np.append(pd.unique(trip_data.loc[:,'from_station_id']),pd.unique(trip_data.loc[:,'to_station_id'])))

temp_list = []
for station in unique_stations:
    start_filter = trip_data.loc[:,'from_station_id'] == station
    end_filter = trip_data.loc[:,'to_station_id'] == station
    start_count = start_filter.sum()
    end_count = end_filter.sum()
    start_min = trip_data.loc[start_filter,'start_time'].min()
    end_min = trip_data.loc[end_filter,'end_time'].min()
    start_max = trip_data.loc[start_filter,'start_time'].max()
    end_max = trip_data.loc[end_filter,'end_time'].max()
    temp_list.append((station, start_count, end_count, start_min, end_min, start_max, end_max))

column_list = ['station_id', 'start_count', 'end_count', 'start_min', 'end_min', 'start_max', 'end_max']
station_data = pd.DataFrame(temp_list, columns=column_list)

In [27]:
# We will send this to csv for use in Excel.
station_data.to_csv('./Supporting_Workfiles/initial_station_data.csv')

See supporting workfile 'Station Analysis.xlsx' for details on how the Divvy-provided data was worked in.<br>

The 565 stations included for the remainder of the analysis were selected as follows:<br>
    1. Active as of the 2017 Q3/Q4 data
    2. Active since at least 2016 to ensure sufficient quantity of data
    3. Bike capacity greater than zero
    4. Not located in Oak Park (discontinuing Divvy operations in 2018)
    
We may now read back in the station data.<br>

**If the kernel has restarted, you may omit running the cells above in this section (beginning at "Create a DataFrame shell for aggregating trip data and weather data") and skip to the cell below.**

In [3]:
columns = [
    'id',
    'name',
    'city',
    'latitude',
    'longitude',
    'dpcapacity',
    'online_date',
    'start_count',
    'end_count',
    'start_min',
    'end_min',
    'start_max',
    'end_max',
    'first_day_operational'
]

data_types = {
    'id':np.int64,
    'name':object,
    'city':object,
    'latitude':np.float64,
    'longitude':np.float64,
    'dpcapacity':np.int64,
    'online_date':object,
    'start_count':np.int64,
    'end_count':np.int64,
    'start_min':object,
    'end_min':object,
    'start_max':object,
    'end_max':object,
    'first_day_operational':object
}

final_station_data = pd.read_csv('./Supporting_Workfiles/final_station_data.csv',
                            header=0, names=columns, index_col='id', dtype=data_types,
                            parse_dates=['online_date','start_min','end_min','start_max','end_max','first_day_operational'],
                            infer_datetime_format=True)

We can now begin creation of the final data file.<br>

We will generate one line, per station, per time bucket, for each day since the station went operational.<br>

It is important to generate the list like this because some stations will not have any trips starting and ending in every possible time bucket. It is important to capture those "zeroes" in the data.

In [4]:
temp_list=[]
for station in final_station_data.index.values:
    start_date = final_station_data.loc[station,'first_day_operational']
    for date in pd.date_range(start_date, '2017-12-31'):
        if date.weekday() <= 4:
            for time in range(5):
                tag = 'ID'+str(station)+'Date'+str(date.date())+'Time'+str(time)
                temp_list.append((station, date, time, tag))
        else:
            for time in range(5,8):
                tag = 'ID'+str(station)+'Date'+str(date.date())+'Time'+str(time)
                temp_list.append((station, date, time, tag))

column_list = ['station', 'date', 'time', 'tag']
final_data = pd.DataFrame(temp_list, columns=column_list)

### Add in trip data

We create DataFrames from the value_counts of the tags from the trip_data, then merge those columns into the final_data.

In [5]:
start_tag_counts = trip_data.loc[:,'start_tag'].value_counts().to_frame('start_tag_count')
end_tag_counts = trip_data.loc[:,'end_tag'].value_counts().to_frame('end_tag_count')

In [6]:
final_data = final_data.merge(start_tag_counts, how='left', left_on='tag', right_index=True)

In [7]:
final_data = final_data.merge(end_tag_counts, how='left', left_on='tag', right_index=True)

In [8]:
# Having merged in the counts, we now do general cleanup to the dataset to make it ready for use.

final_data.loc[:,'start_tag_count'] = final_data.loc[:,'start_tag_count'].fillna(0)
final_data.loc[:,'end_tag_count'] = final_data.loc[:,'end_tag_count'].fillna(0)

final_data.loc[:,'start_tag_count'] = final_data.loc[:,'start_tag_count'].astype(np.int64)
final_data.loc[:,'end_tag_count'] = final_data.loc[:,'end_tag_count'].astype(np.int64)

final_data.loc[:,'bike_differential'] = final_data.loc[:,'end_tag_count'] - final_data.loc[:,'start_tag_count']

final_data = final_data.set_index('tag', drop=True, verify_integrity=True)

In [9]:
final_data.head()

,station,date,time,start_tag_count,end_tag_count,bike_differential
tag,,,,,,
ID2Date2015-05-09Time5,2,2015-05-09,5,0,0,0
ID2Date2015-05-09Time6,2,2015-05-09,6,48,43,-5
ID2Date2015-05-09Time7,2,2015-05-09,7,1,1,0
ID2Date2015-05-10Time5,2,2015-05-10,5,2,2,0
ID2Date2015-05-10Time6,2,2015-05-10,6,27,26,-1


### Add in weather data

Weather data csv's were prepared in Excel to be read in.<br>
See supporting workfiles 'Hourly_Weather_Data.xlsx' and 'Daily_Weather_Normals.xlsx'.<br>
The NOAA documentation has been saved with the supporting workfiles in pdf form.

In [10]:
# First we need to create a day_of_year tag for the daily normal weather and a date_and_time tag for the hourly weather.
final_data.loc[:,'day_of_year'] = final_data.loc[:,'date'].dt.month.astype(str) + '-' + final_data.loc[:,'date'].dt.day.astype(str)
final_data.loc[:,'date_and_time'] = final_data.loc[:,'date'].dt.date.astype(str) + 'Time' + final_data.loc[:,'time'].astype(str)

In [12]:
# Next we load in the daily normal weather data.

columns = [
    'day_of_year',
    'normal_temp_range',
    'normal_temp_ave',
    'normal_temp_max',
    'normal_temp_min'
]

data_types = {
    'day_of_year':object,
    'normal_temp_range':np.float64,
    'normal_temp_ave':np.float64,
    'normal_temp_max':np.float64,
    'normal_temp_min':np.float64
}

daily_weather_normals = pd.read_csv('./Supporting_Workfiles/Daily_Weather_Normals.csv',
                            header=0, names=columns, index_col='day_of_year', dtype=data_types)

In [13]:
# Merge in the daily normal weather data.
final_data = final_data.merge(daily_weather_normals, how='left', left_on='day_of_year', right_index=True)

In [15]:
# Next we load in the current weather data for each time bucket.

columns = [
    'date_and_time',
    'current_temp',
    'current_dew_point',
    'current_humidity',
    'current_wind_speed',
    'current_pressure',
    'current_precipitation'
]

data_types = {
    'date_and_time':object,
    'current_temp':np.float64,
    'current_dew_point':np.float64,
    'current_humidity':np.float64,
    'current_wind_speed':np.float64,
    'current_pressure':np.float64,
    'current_precipitation':np.float64
}

current_weather_data = pd.read_csv('./Supporting_Workfiles/Hourly_Weather_Data.csv',
                            header=0, names=columns, index_col='date_and_time', dtype=data_types)

In [19]:
# Merge in the current weather data.
final_data = final_data.merge(current_weather_data, how='left', left_on='date_and_time', right_index=True)

### Clean up and export to .csv

The final data file will be called divvy_data.<br>
The only changes to make before exporting are to add in any additional formula-based columns and drop unnecessary columns.

In [27]:
# Adding in a total_activity column, the sum of rides beginning and ending at the station.
final_data.loc[:,'total_activity'] = final_data.loc[:,'start_tag_count'] + final_data.loc[:,'end_tag_count']

# Dropping unnecessary 'tag' columns used for merges.
divvy_data = final_data.drop(['day_of_year', 'date_and_time'], axis=1)

In [32]:
# Finally, we export the csv file to be used in the analysis.
divvy_data.to_csv('./Supporting_Workfiles/divvy_data.csv')